## TensorFlow Data Input Pipeline + AlexNet CNN 
This notebook servers has a simple example of loading the dataset into TensorFlow for better processing and optimization.
We will process and visualize the dataset and later build a classification model on it. The dataset contains additional data such as segmentations and bounding boxes which are useful in bulding more robust models but we are not going to utilize that for this notebook.

We get this note from the data description that some of the DICOM files are JPEG compressed. You may require additional resources to read the pixel array of these files, such as GDCM and pylibjpeg. WE will install this dependencies

In [3]:
import sys

!{sys.executable} -m pip install '../input/cspine-helper/pylibjpeg-1.4.0-py3-none-any.whl' -q
!{sys.executable} -m pip install '../input/cspine-helper/pylibjpeg_libjpeg-1.3.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl' -q
!{sys.executable} -m pip install '../input/cspine-helper/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl' -q

## Handle imports

In [4]:
import os 
import pathlib
import glob 
from tqdm import tqdm 
import gdcm

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import pydicom

In [ ]:
sns.set()

In [ ]:
## Parameters
EPOCHS = 10
BATCH_SIZE = 16
IMAGE_SIZE = (512, 512)
SEED = 42

In [ ]:
# set seed
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Data  EDA and Processing

In [ ]:
 # the input root folder 
DATA_DIR = "../input/rsna-2022-cervical-spine-fracture-detection/"

In [ ]:
# lets list the contents inside the root folder
os.listdir(DATA_DIR)

In [ ]:
# look at what is in the train.csv
train_df = pd.read_csv(DATA_DIR + "train.csv")
train_df.head(10)

In [ ]:
train_df.size

In [ ]:
# how many unique study instances do we have
train_df.StudyInstanceUID.nunique()

The train dataframe has metadata for the each study instance, c1..c7 are the cervical vertebrae planes and the values in the rows states whether it is fractured or not.

We will use this for our classification model.

In [ ]:
# a little deeper inside the train images
os.listdir(DATA_DIR + "train_images")[:5]

we can see that the train images folder has other subfolders with the study id has its name. Each study can contain several instances with several frames and for this case slices in dicom format.

In [ ]:
study_instance = "1.2.826.0.1.3680043.17625"
# list the first 5 frames in a select study instance
os.listdir(DATA_DIR + f"train_images/{study_instance}")[:5]

In [ ]:
# select all the dicom files in the study instance
img_list = glob.glob(DATA_DIR + f"/train_images/{study_instance}/*.dcm")
len(img_list)

### Data Loading Functionalities 

In [ ]:
def load_dicom(path):
    """
    reads a dicom file and loads the image array inside it
    inputs:
        path: the path of the required dicom file
    returns:
        data: image pixel arrays
    """
    img=pydicom.dcmread(path)
    data=img.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
def data_generator():
    """
    a function that will load the dataset from a list of image paths
    """
    for path in img_list:
        data = load_dicom(path)
        yield data  # return the data has generator

In [ ]:
# lets define a tensorflow dataset variable that will use the generator to get the image data
# this is efficient beacuse it will only load the data into memory when needed
train_dataset = tf.data.Dataset.from_generator(data_generator, (tf.uint8))

In [ ]:
# a quick look of the dataset contents
for i in train_dataset.take(1):
    print(i.shape)
    print(type(i))

### Data Visualization

In [ ]:
def show_single(img, cmap="gray"):
    """
    plots a single image
    """
    plt.imshow(img, cmap=cmap)
    plt.axis("off")

In [ ]:
show_single(i)

In [ ]:
def show_batch(cmap="gray"):
    """
    visualizes a batch of images
    """
    plt.figure(figsize=(16, 12))
    for i, img in enumerate(train_dataset.take(20)):  # iterate through the dataset
        plt.subplot(4, 5, i+1)
        show_single(img, cmap=cmap)
    plt.show()

#### A look of the images using different color maps

In [ ]:
show_batch(cmap="gray")

In [ ]:
show_batch(cmap="bone")

In [ ]:
show_batch(cmap="inferno")

We have so far loaded the images, but it is not ready for training. We need to map the labels from the train_df and as seen earlier, we have 2019 unique study instances. Hence, we have to create a nested loop to iterate through all the study instances and the dicom files inside each study instance.

In [ ]:
# lets modify the data generator, use 10 study instances
def data_generator():
    for i, study_instance in enumerate(train_df.StudyInstanceUID[:5]):
        for dcm in os.listdir(DATA_DIR + f"train_images/{study_instance}"):
            train_labels = []
            path = DATA_DIR + f"train_images/{study_instance}/{dcm}"
            
            img = load_dicom(path)
            
            # resize each image into a shape of (512, 512)
            img = np.resize(img, (512, 512))
            #  normalize image
            img = img / 255.0
            # convert from gray scale to rgb, this will be helpful incase we want to use pretrained models
            img = tf.expand_dims(img, axis=-1)
            img = tf.image.grayscale_to_rgb(img)
            
            train_labels.extend([
                train_df.loc[i, "C1"],
                train_df.loc[i, "C2"],
                train_df.loc[i, "C3"],
                train_df.loc[i, "C4"],
                train_df.loc[i, "C5"],
                train_df.loc[i, "C6"],
                train_df.loc[i, "C7"],
                train_df.loc[i, "patient_overall"] # end with patient overall
            ])
            yield img, train_labels

In [ ]:
train_data = tf.data.Dataset.from_generator(data_generator, (tf.float32, tf.int8))

In [ ]:
for img, label in train_data.take(1):
    print(img.shape)
    print(label.shape)
    print(label)

In [ ]:
# visualize the image once again
show_single(img, cmap="gray")

Now we have our one-hot encoded labels, the last thing is to prepare the dataset for training by batching, catching and shuffling

### Split into train and validation

In [ ]:
# we first need to know the number of data points we are dealing with
img_count = 0
for _, _ in enumerate(train_df.StudyInstanceUID[:5]):
    for _ in os.listdir(DATA_DIR + f"train_images/{study_instance}"):
        img_count += 1
print(img_count)

In [ ]:
val_size = int(img_count * 0.2)
train_data = train_data.skip(val_size)
val_data = train_data.take(val_size)

In [ ]:
def configure_for_performance(data):
    data = data.cache()
#     data = data.shuffle(buffer_size=300)
    data = data.batch(16)
    data = data.prefetch(buffer_size=tf.data.AUTOTUNE)
    return data

In [ ]:
train_data = configure_for_performance(train_data)
val_data = configure_for_performance(val_data)

# Modelling

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Flatten

In [ ]:
# Define Alex Net model
def alex_net():
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(512,512,3), kernel_size=(11,11),\
     strides=(4,4), padding='valid', activation="relu"))
    # Pooling 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid', activation="relu"))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation="relu"))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid', activation="relu"))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', activation="relu"))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Passing it to a dense layer
    model.add(Flatten())
    # 1st Dense Layer
    model.add(Dense(4096, input_shape=(512*512*3,), activation="relu"))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 2nd Dense Layer
    model.add(Dense(4096, activation="relu"))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Dense Layer
    model.add(Dense(1000, activation="relu"))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer with 8 probability classes
    model.add(Dense(8, activation="softmax"))
    return model

In [ ]:
model = alex_net()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()]
             )

In [ ]:
# training
history = model.fit(train_data, validation_data=val_data,
                   epochs=EPOCHS)

In [ ]:
# visualize training 
def viz_loss(history):
    train_loss = history["loss"]
    val_loss = history["val_loss"]
    iters = [i for i in range(EPOCHS)]
    
    plt.plot(iters, train_loss, label="Training Loss")
    plt.plot(iters, val_loss, label="Validation Loss")
    plt.title("A plot of Loss against number of iterations")
    plt.legend()
    plt.show()
    
def viz_acc(history):
    train_loss = history["categorical_accuracy"]
    val_loss = history["val_categorical_accuracy"]
    iters = [i for i in range(EPOCHS)]
    
    plt.plot(iters, train_loss, label="Training Accuracy")
    plt.plot(iters, val_loss, label="Validation Accuracy")
    plt.title("A plot of Accuracy against number of iterations")
    plt.legend()
    plt.show()

In [ ]:
viz_loss(history.history)
viz_acc(history.history)

# Submission

In [ ]:
# prep test data for submission
test_df = pd.read_csv(DATA_DIR + "test.csv")
test_df.head()

In [ ]:
global test_ids
test_ids = []
def test_data_generator():
    for study_instance in os.listdir(DATA_DIR + f"test_images"):
        for dcm in os.listdir(DATA_DIR + f"test_images/{study_instance}"):
            path = DATA_DIR + f"test_images/{study_instance}/{dcm}"
            img = load_dicom(path)
            
            # resize each image into a shape of (512, 512)
            img = np.resize(img, (512, 512))
            #  normalize image
            img = tf.cast(img, tf.float32) / 255.0
            # convert from gray scale to rgb, this will be helpful incase we want to use pretrained models
            img = tf.expand_dims(img, axis=-1)
            img = tf.image.grayscale_to_rgb(img)
            test_ids.append(study_instance)
            yield img

In [ ]:
test_data = tf.data.Dataset.from_generator(test_data_generator, tf.float32).batch(1)

In [ ]:
# make predictions
preds = []
for img in tqdm(test_data):
    preds.append(model.predict(img)[0])
preds = np.array(preds)

In [ ]:
assert len(test_ids) == len(preds)

In [ ]:
result = pd.DataFrame(columns = ["StudyInstanceUID", 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'patient_overall'])

In [ ]:
for i in tqdm(range(len(test_ids))):
    result.loc[i, 'StudyInstanceUID'] = test_ids[i]
    rows = preds[i].round(3)
    result.loc[i, 'C1'] = rows[0]
    result.loc[i, 'C2'] = rows[1]
    result.loc[i, 'C3'] = rows[2]
    result.loc[i, 'C4'] = rows[3]
    result.loc[i, 'C5'] = rows[4]
    result.loc[i, 'C6'] = rows[5]
    result.loc[i, 'C7'] = rows[6]
    result.loc[i, 'patient_overall'] = rows[7]

In [ ]:
result.head()

In [ ]:
# means = result[['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].mean().to_dict()
# sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")
# sample_submission.head()
# sample_submission.to_csv("submission.csv", index="false")

In [ ]:
#make submission corresponding to the submission format
# idxs = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'patient_overall']
# sub = []
# for i in tqdm(range(len(test_ids))):
#     for j, el in enumerate(idxs):
#         sub.append([result.loc[i].StudyInstanceUID + f"_{el}", result.loc[0][j+1]])

In [ ]:
means = result[['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].mean().to_dict()
print(means)

In [ ]:
test_df['fractured'] = test_df['prediction_type'].map(means)
test_df[['row_id','fractured']].to_csv('submission.csv', index=False, float_format='%.1g')

In [ ]:
!cat submission.csv

# Conclusion 
We have seen how to process the dicom files and transform the dataset into tensorflow format. However, we didn't use all the dataset and the cpu memory was filling up easily. We could do some optimizations to train with all the images.

## What to do next
1. Handle JPEG dicom format correctly (had some few errors)
2. Train will all the images
3. Use segmentations and bounding boxes
4. Perform cross validation